<a href="https://colab.research.google.com/github/hossein20s/tutorial/blob/master/LSTM_in_credit_card_fraud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import tensorflow as tf

print(tf.__version__)

1.13.1


In [16]:
import sys
sys.path.append('/content/gdrive/My Drive/python-lib')

from google.colab import drive
drive.mount('/content/gdrive')

persist_path = "gdrive/My Drive/models/credit-card-fraud"


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [17]:
import pandas as pd

data = pd.read_csv('/content/gdrive/My Drive//data/creditcard.csv.zip', compression='zip', skiprows=[0], header=None)

features = data.iloc[:, 1:30]
labels = data.iloc[:, -1]

print(data[:2])
print(features[:2])
print(labels[:2])

    0         1         2         3         4         5         6         7   \
0  0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1  0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   

         8         9  ...        21        22        23        24        25  \
0  0.098698  0.363787 ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425 ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   

         26        27        28      29  30  
0 -0.189115  0.133558 -0.021053  149.62   0  
1  0.125895 -0.008983  0.014724    2.69   0  

[2 rows x 31 columns]
         1         2         3         4         5         6         7   \
0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   

         8         9         10   ...          20        21        22  \
0  0.098698  0.363787  0.090794   ...    0.251412 -0.018307  

In [0]:
from sklearn.model_selection import train_test_split

train_x,test_x,train_y,test_y = train_test_split(features, labels, test_size=0.2, shuffle=False, random_state=42)

In [0]:
from tensorflow.contrib import rnn
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

epochs = 2
batch_size = 35
n_classes = 1
n_units = 200
n_features = 29


xplaceholder= tf.placeholder('float',[None,n_features])
yplaceholder = tf.placeholder('float')



def recurrent_neural_network_model():
    layer ={ 'weights': tf.Variable(tf.random_normal([n_units, n_classes])),'bias': tf.Variable(tf.random_normal([n_classes]))}

    x = tf.split(xplaceholder, n_features, 1)

    lstm_cell = rnn.BasicLSTMCell(n_units)
    
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    
    output = tf.matmul(outputs[-1], layer['weights']) + layer['bias']

    return output






In [0]:
xplaceholder= tf.placeholder('float',[None,n_features])
yplaceholder = tf.placeholder('float')

# for TF 2.0
#xplaceholder= tf.Variable(n_features)
#yplaceholder = tf.Variable('float')


def train_neural_network(model_dir, X_train, y_train, X_test, y_test):
    logit = recurrent_neural_network_model()
    logit = tf.reshape(logit, [-1])

    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit, labels=yplaceholder))
    optimizer = tf.train.AdamOptimizer().minimize(cost)

    saver = tf.train.Saver()
      
    with tf.Session() as sess:

        tf.global_variables_initializer().run()
        tf.local_variables_initializer().run()

        for epoch in range(epochs):
            epoch_loss = 0

            i = 0
            for i in range(int(len(X_train) / batch_size)):

                start = i
                end = i + batch_size

                batch_x = np.array(X_train[start:end])
                batch_y = np.array(y_train[start:end])
                
                _, c = sess.run([optimizer, cost], feed_dict={xplaceholder: batch_x, yplaceholder: batch_y})
                epoch_loss += c
                i += batch_size

            print('Epoch', epoch, 'completed out of', epochs, 'loss:', epoch_loss)

        saver.save(sess, model_dir + '/fraud.ckpt')
        
        pred = tf.round(tf.nn.sigmoid(logit)).eval({xplaceholder: np.array(X_test), yplaceholder: np.array(y_test)})
        f1 = f1_score(np.array(y_test), pred, average='macro')
        accuracy=accuracy_score(np.array(y_test), pred)
        recall = recall_score(y_true=np.array(y_test), y_pred= pred)
        precision = precision_score(y_true=np.array(y_test), y_pred=pred)
        print("F1 Score:", f1)
        print("Accuracy Score:",accuracy)
        print("Recall:", recall)
        print("Precision:", precision)

#clear you computational graph
#tf.reset_default_graph()
train_neural_network(persist_path, train_x, train_y, test_x, test_y)



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.
('Epoch', 0, 'completed out of', 2, 'loss:', 67.73385608999104)
('Epoch', 1, 'completed out of', 2, 'loss:', 23.13725229981863)
('F1 Score:', 0.6049982068679287)
('Accuracy Score:', 0.9964713317650363)
('Recall:', 0.36)
('Precision:', 0.15)


In [21]:
from tensorflow.contrib import rnn
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score


def test_network(model_dir, batch_size, y_test):
  xplaceholder= tf.placeholder('float',[None,n_features])
  yplaceholder = tf.placeholder('float')
  #saver = tf.train.import_meta_graph(model_dir + '/fraud.ckpt.meta')

  logit = recurrent_neural_network_model()
  logit = tf.reshape(logit, [-1])
  
    
  saver = tf.train.Saver()
    
  test_acc = []
  with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint(model_dir))
    
    pred = tf.round(tf.nn.sigmoid(logit)).eval({xplaceholder: np.array(test_x), yplaceholder: np.array(test_y)})                                         
    f1 = f1_score(np.array(y_test), pred, average='macro')
    accuracy=accuracy_score(np.array(y_test), pred)
    recall = recall_score(y_true=np.array(y_test), y_pred= pred)
    precision = precision_score(y_true=np.array(y_test), y_pred=pred)

    print("F1 Score:", f1)
    print("Accuracy Score:",accuracy)
    print("Recall:", recall)
    print("Precision:", precision)        
        

#with tf.Graph().as_default():
tf.reset_default_graph()
test_network(persist_path, batch_size, test_y)

ValueError: ignored